# N-Gram 模型
上一节课，我们讲了词嵌入以及词嵌入是如何得到的，现在我们来讲讲词嵌入如何来训练语言模型，首先我们介绍一下 N-Gram 模型的原理和其要解决的问题。

对于一句话，单词的排列顺序是非常重要的，所以我们能否由前面的几个词来预测后面的几个单词呢，比如 'I lived in France for 10 years, I can speak _' 这句话中，我们能够预测出最后一个词是 French。

对于一句话 T，其由 $w_1, w_2, \cdots, w_n$ 这 n 个词构成，

$$
P(T) = P(w_1)P(w_2 | w_1)P(w_3 |w_2 w_1) \cdots P(w_n |w_{n-1} w_{n-2}\cdots w_2w_1)
$$

我们可以再简化一下这个模型，比如对于一个词，它并不需要前面所有的词作为条件概率，也就是说一个词可以只与其前面的几个词有关，这就是马尔科夫假设。

对于这里的条件概率，传统的方法是统计语料中每个词出现的频率，根据贝叶斯定理来估计这个条件概率，这里我们就可以用词嵌入对其进行代替，然后使用 RNN 进行条件概率的计算，然后最大化这个条件概率不仅修改词嵌入，同时能够使得模型可以依据计算的条件概率对其中的一个单词进行预测。

下面我们直接用代码进行说明

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

CONTEXT_SIZE = 2 # 依据的单词数
EMBEDDING_DIM = 10 # 词向量的维度
# 我们使用莎士比亚的诗
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()

这里的 `CONTEXT_SIZE` 表示我们希望由前面几个单词来预测这个单词，这里使用两个单词，`EMBEDDING_DIM` 表示词嵌入的维度。

接着我们建立训练集，便利整个语料库，将单词三个分组，前面两个作为输入，最后一个作为预测的结果。

In [ ]:
 trigram = [((test_sentence[i], test_sentence[i+1]), test_sentence[i+2]) 
            for i in range(len(test_sentence)-2)]

In [ ]:
# 总的数据量
len(trigram)

In [ ]:
# 取出第一个数据看看
trigram[0]

In [ ]:
# 建立每个词与数字的编码，据此构建词嵌入
vocb = set(test_sentence) # 使用 set 将重复的元素去掉
word_to_idx = {word: i for i, word in enumerate(vocb)}
idx_to_word = {word_to_idx[word]: word for word in word_to_idx}

In [ ]:
word_to_idx

从上面可以看到每个词都对应一个数字，且这里的单词都各不相同

接着我们定义模型，模型的输入就是前面的两个词，输出就是预测单词的概率

In [ ]:
import tensorflow as tf
import tensorflow.contrib.slim as slim

In [ ]:
def n_gram(inputs, vocab_size, context_size=CONTEXT_SIZE, n_dim=EMBEDDING_DIM, scope='n-gram', reuse=None):
    with tf.variable_scope(scope, reuse=reuse):
        with tf.device('/cpu:0'):
            embeddings = tf.get_variable('embeddings', shape=[vocab_size, n_dim], initializer=tf.random_uniform_initializer)
        embed = tf.nn.embedding_lookup(embeddings, inputs)
        
        net = tf.reshape(embed, (1, -1))
        net = slim.fully_connected(net, vocab_size, activation_fn=None, scope='classification')
        
        return net

In [ ]:
input_ph = tf.placeholder(dtype=tf.int64, shape=[2 ], name='input')
label_ph = tf.placeholder(dtype=tf.int64, shape=[1,], name='label')

net = n_gram(input_ph, len(word_to_idx))

In [ ]:
loss = tf.losses.sparse_softmax_cross_entropy(label_ph, net, scope='loss')

In [ ]:
opt = tf.train.MomentumOptimizer(1e-2, 0.9)
train_op = opt.minimize(loss)

In [ ]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [ ]:
for e in range(100):
    train_loss = 0
    for word, label in trigram[:100]:
        word = [word_to_idx[i] for i in word]
        label = [word_to_idx[label]]
        
        _, curr_loss = sess.run([train_op, loss], feed_dict={input_ph: word, label_ph: label})
        train_loss += curr_loss
    
    if (e + 1) % 20 == 0:
        print('Epoch: {}, Loss: {:.6f}'.format(e + 1, train_loss / 100))

In [ ]:
# 测试一下结果
word, label = trigram[19]
print('input: {}'.format(word))
print('label: {}'.format(label))
print()
word = [word_to_idx[i] for i in word]
out = sess.run(net, feed_dict={input_ph: word})
pred_label_idx = out[0].argmax()
predict_word = idx_to_word[pred_label_idx]
print('real word is {}, predicted word is {}'.format(label, predict_word))

In [ ]:
word, label = trigram[75]
print('input: {}'.format(word))
print('label: {}'.format(label))
print()
word = [word_to_idx[i] for i in word]
out = sess.run(net, feed_dict={input_ph: word})
pred_label_idx = out[0].argmax()
predict_word = idx_to_word[pred_label_idx]
print('real word is {}, predicted word is {}'.format(label, predict_word))

可以看到网络在训练集上基本能够预测准确，不过这里样本太少，特别容易过拟合。

下一次课我们会讲一讲 RNN 如何应用在自然语言处理中